# Лабораторная работа 4: Метод k-ближайших соседей

## Реализация без использования библиотек машинного обучения, только NumPy и Pandas

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import confusion_matrix

# Настройка отображения графиков
%matplotlib inline
plt.rcParams["figure.figsize"] = (10, 6)
sns.set_style("whitegrid")

## 1. Загрузка и предварительная обработка данных

In [ ]:
# Загрузка данных
df = pd.read_csv("data/WineDataset.csv")

print("1. ПРЕДВАРИТЕЛЬНАЯ ОБРАБОТКА ДАННЫХ")
print("=" * 50)

# Проверка на отсутствующие значения
print("Проверка отсутствующих значений:")
print(df.isnull().sum())
print()

# Информация о датасете
print("Информация о датасете:")
print(df.info())
print()

# Масштабирование признаков (стандартизация)
scaler = StandardScaler()
X = df.drop("Wine", axis=1)
y = df["Wine"]
X_scaled = scaler.fit_transform(X)

print("Размерность данных после масштабирования:", X_scaled.shape)
print()

## 2. Статистика и визуализация данных

In [ ]:
print("2. СТАТИСТИКА И ВИЗУАЛИЗАЦИЯ ДАННЫХ")
print("=" * 50)

# Базовая статистика
print("Статистика датасета:")
print(df.describe())
print()

In [ ]:
# Визуализация распределения классов
plt.figure(figsize=(10, 6))
sns.countplot(x="Wine", data=df)
plt.title("Распределение классов вина")
plt.show()

### 2.1. Визуализация распределения признаков

In [ ]:
print("2.1. ВИЗУАЛИЗАЦИЯ РАСПРЕДЕЛЕНИЯ ПРИЗНАКОВ")
print("=" * 50)

# Выбор 9 признаков для визуализации
selected_features = [
    'Alcohol', 'Malic Acid', 'Ash', 
    'Alcalinity of ash', 'Magnesium', 'Total phenols',
    'Flavanoids', 'Color intensity', 'Proline'
]

# Создание сетки графиков 3x3
fig, axes = plt.subplots(3, 3, figsize=(15, 12))
fig.suptitle('Гистограммы распределения признаков вина', fontsize=16, fontweight='bold')

# Построение гистограмм для каждого признака
for i, feature in enumerate(selected_features):
    row = i // 3
    col = i % 3
    
    # Гистограмма с помощью matplotlib
    axes[row, col].hist(df[feature], bins=15, color='skyblue', edgecolor='black', alpha=0.7)
    
    # Настройка внешнего вида
    axes[row, col].set_title(f'Распределение {feature}', fontweight='bold')
    axes[row, col].set_xlabel(feature)
    axes[row, col].set_ylabel('Частота')
    axes[row, col].grid(True, alpha=0.3)
    
    # Добавление вертикальной линии для среднего значения
    mean_val = df[feature].mean()
    axes[row, col].axvline(mean_val, color='red', linestyle='--', linewidth=2, label=f'Среднее: {mean_val:.2f}')
    axes[row, col].legend()

plt.tight_layout()
plt.show()

In [ ]:
# 3D визуализация первых трех признаков
fig = plt.figure(figsize=(13, 16))
ax = fig.add_subplot(111, projection="3d")

colors = ["red", "blue", "green"]
for i in range(1, 4):
    mask = y == i
    ax.scatter(
        X_scaled[mask, 0],
        X_scaled[mask, 1],
        X_scaled[mask, 2],
        c=colors[i - 1],
        label=f"Class {i}",
        alpha=0.7,
    )

ax.set_xlabel("Feature 1 (Alcohol)")
ax.set_ylabel("Feature 2 (Malic Acid)")
ax.set_zlabel("Feature 3 (Ash)")
ax.set_title("3D визуализация признаков вина")
ax.legend()
plt.show()

## 3. Реализация k-NN с нуля

In [ ]:
class KNN:
    def __init__(self, k=3):
        self.k = k

    def fit(self, X, y):
        self.X_train = X
        self.y_train = y

    def predict(self, X):
        predictions = [self._predict(x) for x in X]
        return np.array(predictions)

    def _euclidean_distance(self, x1, x2):
        return np.sqrt(np.sum((x1 - x2) ** 2))

    def _predict(self, x):
        # Вычисление расстояний до всех точек обучающей выборки
        distances = [self._euclidean_distance(x, x_train) for x_train in self.X_train]

        # Получение k ближайших соседей
        k_indices = np.argsort(distances)[: self.k]
        k_nearest_labels = [self.y_train[i] for i in k_indices]

        # Голосование большинством
        most_common = np.bincount(k_nearest_labels).argmax()
        return most_common

    def accuracy(self, y_true, y_pred):
        return np.sum(y_true == y_pred) / len(y_true)

## 4. Построение моделей с различными наборами признаков

In [ ]:
print("4. ПОСТРОЕНИЕ МОДЕЛЕЙ С РАЗЛИЧНЫМИ НАБОРАМИ ПРИЗНАКОВ")
print("=" * 50)

# Разделение на обучающую и тестовую выборки
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.3, random_state=42)

y_train = y_train.values
y_test = y_test.values

# Модель 1: Случайный выбор признаков
print("МОДЕЛЬ 1: СЛУЧАЙНЫЙ ВЫБОР ПРИЗНАКОВ")
print("-" * 40)

# Выбор 5 случайных признаков
np.random.seed(42)
random_features = np.random.choice(X_scaled.shape[1], size=5, replace=False)
print(f"Случайно выбранные признаки: {random_features}")

X_train_random = X_train[:, random_features]
X_test_random = X_test[:, random_features]

# Модель 2: Фиксированный набор признаков
print("\nМОДЕЛЬ 2: ФИКСИРОВАННЫЙ НАБОР ПРИЗНАКОВ")
print("-" * 40)

# Выбор фиксированных признаков (первые 5)
fixed_features = [0, 1, 2, 3, 4]  # Alcohol, Malic Acid, Ash, Alcalinity of ash, Magnesium
print(f"Фиксированные признаки: {fixed_features}")

X_train_fixed = X_train[:, fixed_features]
X_test_fixed = X_test[:, fixed_features]

## 5. Оценка моделей при различных значениях k

In [ ]:
print("5. ОЦЕНКА МОДЕЛЕЙ ПРИ РАЗЛИЧНЫХ ЗНАЧЕНИЯХ k")
print("=" * 50)

k_values = [3, 5, 10, 15]


def evaluate_model(X_train, X_test, y_train, y_test, model_name):
    print(f"\n{model_name}")
    print("-" * 30)

    results = {}

    for k in k_values:
        # Создание и обучение модели
        knn = KNN(k=k)
        knn.fit(X_train, y_train)

        # Предсказание
        y_pred = knn.predict(X_test)

        # Вычисление точности
        accuracy = knn.accuracy(y_test, y_pred)
        results[k] = accuracy

        print(f"k = {k}, Точность: {accuracy:.4f}")

        # Матрица ошибок
        cm = confusion_matrix(y_test, y_pred)

        plt.figure(figsize=(6, 5))
        sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
        plt.title(f"Матрица ошибок (k={k}, {model_name})")
        plt.ylabel("Истинные значения")
        plt.xlabel("Предсказанные значения")
        plt.show()

    return results


# Оценка моделей
results_random = evaluate_model(
    X_train_random,
    X_test_random,
    y_train,
    y_test,
    "Модель 1: Случайные признаки",
)
results_fixed = evaluate_model(
    X_train_fixed,
    X_test_fixed,
    y_train,
    y_test,
    "Модель 2: Фиксированные признаки",
)

In [ ]:
# Сравнение результатов
plt.figure(figsize=(10, 6))
plt.plot(k_values, [results_random[k] for k in k_values], "o-", label="Случайные признаки")
plt.plot(k_values, [results_fixed[k] for k in k_values], "s-", label="Фиксированные признаки")
plt.xlabel("Значение k")
plt.ylabel("Точность")
plt.title("Сравнение точности моделей при различных k")
plt.legend()
plt.grid(True)
plt.show()

## 6. Выводы

In [ ]:
print("\n6. ВЫВОДЫ")
print("=" * 50)
print("\nЛучшая точность достигнута при:")
best_k_random = max(results_random, key=results_random.get)
best_k_fixed = max(results_fixed, key=results_fixed.get)
print(
    f"- Модель 1 (случайные признаки): k={best_k_random}, точность={results_random[best_k_random]:.4f}"
)
print(
    f"- Модель 2 (фиксированные признаки): k={best_k_fixed}, точность={results_fixed[best_k_fixed]:.4f}"
)